In [7]:
# First, we import the libaries
import quforge.quforge as qf
from quforge.quforge import State as State

In [8]:
device = 'cpu'

dim = 3 #dimension of the qudit
wires = 2 #number of qudits

In [9]:
#define the oracle
def oracle(circuit, register, marked_state): 
    basis = qf.zeros(9, 1, device=device)
    basis[8][0] = 1.0
    target_state = basis @ basis.T

    U = qf.kron(qf.eye(dim), qf.eye(dim)) - 2*target_state
    circuit.Custom(M=U)

#define the Grover diffusion step
def grover_diffusion(circuit, register): 
    basis = qf.ones(dim**wires, 1, device=device)/((dim**wires)**0.5)
    U = 2*basis@basis.T - qf.eye(9)
    circuit.Custom(M=U)

#define circuit
circuit = qf.Circuit(dim=dim, wires=wires, device=device)
register = [i for i in range(wires)]

In [10]:
#apply Haddamard on each qudit
circuit.H(index=register)

#apply Grover's algorithm
num_iterations = int(round((2 ** 3) ** 0.5)) 
for i in range(1): 
    # Apply the oracle
    oracle(circuit, register, '22') 

    #Apply the diffusion
    grover_diffusion(circuit, register)

#initial state
state = State('0-0', dim=dim)

#Apply the circuit on the initial state
output = circuit(state)
print(output)

tensor([[0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.1852+0.j],
        [0.8519+0.j]])


In [12]:
#Measure the first N-1 qudits
histogram, p = qf.measure(output, index=register, dim=dim, shots=1000)

print(histogram)
print(p)

{'00': 33, '01': 42, '02': 33, '10': 33, '11': 35, '12': 31, '20': 37, '21': 31, '22': 725}
tensor([0.0343, 0.0343, 0.0343, 0.0343, 0.0343, 0.0343, 0.0343, 0.0343, 0.7257])
